In [1]:
import numpy as np
from keras.applications.inception_v3 import InceptionV3
from keras.models import Sequential, load_model, Model
from keras.layers import Input, Dropout, Flatten, Conv2D, MaxPooling2D, Dense, Activation, GlobalAveragePooling2D
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard
import itertools

Using TensorFlow backend.


In [2]:
# all images will be converted to this size
ROWS = 256
COLS = 256
CHANNELS = 3

In [3]:
train_image_generator = ImageDataGenerator(horizontal_flip=True, rescale=1./255, rotation_range=45)
test_image_generator = ImageDataGenerator(horizontal_flip=False, rescale=1./255, rotation_range=0)

train_generator = train_image_generator.flow_from_directory('train', target_size=(ROWS, COLS), class_mode='categorical')
test_generator = test_image_generator.flow_from_directory('test', target_size=(ROWS, COLS), class_mode='categorical')

Found 5994 images belonging to 200 classes.
Found 5794 images belonging to 200 classes.


In [4]:
# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# add a fully-connected layer
x = Dense(1024, activation='relu')(x)
out_layer = Dense(200, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=out_layer)

In [5]:
# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, None, None, 3 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, None, None, 3 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [6]:
tensorboard = TensorBoard(log_dir='./logs')

model.fit_generator(train_generator, steps_per_epoch=32, epochs=100, callbacks=[tensorboard], verbose=2)

Epoch 1/100
 - 27s - loss: 5.6877 - acc: 0.0098
Epoch 2/100
 - 25s - loss: 5.2656 - acc: 0.0137
Epoch 3/100
 - 26s - loss: 5.1166 - acc: 0.0225
Epoch 4/100
 - 26s - loss: 4.8536 - acc: 0.0420
Epoch 5/100
 - 26s - loss: 4.6319 - acc: 0.0703
Epoch 6/100
 - 26s - loss: 4.2571 - acc: 0.1152
Epoch 7/100
 - 26s - loss: 4.0129 - acc: 0.1406
Epoch 8/100
 - 26s - loss: 3.8202 - acc: 0.1523
Epoch 9/100
 - 26s - loss: 3.6958 - acc: 0.1562
Epoch 10/100
 - 25s - loss: 3.5866 - acc: 0.1702
Epoch 11/100
 - 26s - loss: 3.4458 - acc: 0.1982
Epoch 12/100
 - 26s - loss: 3.2257 - acc: 0.2148
Epoch 13/100
 - 26s - loss: 3.0845 - acc: 0.2402
Epoch 14/100
 - 25s - loss: 3.0548 - acc: 0.2653
Epoch 15/100
 - 26s - loss: 3.0389 - acc: 0.2598
Epoch 16/100
 - 26s - loss: 2.9203 - acc: 0.2832
Epoch 17/100
 - 26s - loss: 2.8910 - acc: 0.2656
Epoch 18/100
 - 26s - loss: 2.7140 - acc: 0.2949
Epoch 19/100
 - 26s - loss: 2.6081 - acc: 0.3418
Epoch 20/100
 - 26s - loss: 2.6706 - acc: 0.3330
Epoch 21/100
 - 26s - loss: 2

In [7]:
print(model.evaluate_generator(test_generator, steps=5000))

[2.3260338047121207, 0.44336327658772534]


In [8]:
# unfreeze all layers for more training
for layer in model.layers:
    layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

model.fit_generator(train_generator, steps_per_epoch=32, epochs=100)

Epoch 1/100
32/32 [==============================] - 46s 1s/step - loss: 1.3258 - acc: 0.6212
Epoch 2/100
32/32 [==============================] - 27s 855ms/step - loss: 1.1749 - acc: 0.6816
Epoch 3/100
32/32 [==============================] - 27s 855ms/step - loss: 1.1220 - acc: 0.6758
Epoch 4/100
32/32 [==============================] - 28s 863ms/step - loss: 1.0958 - acc: 0.7109
Epoch 5/100
32/32 [==============================] - 27s 843ms/step - loss: 1.0043 - acc: 0.7129
Epoch 6/100
32/32 [==============================] - 27s 855ms/step - loss: 0.9802 - acc: 0.7100
Epoch 7/100
32/32 [==============================] - 27s 846ms/step - loss: 0.8731 - acc: 0.7432
Epoch 8/100
32/32 [==============================] - 27s 852ms/step - loss: 0.8452 - acc: 0.7646
Epoch 9/100
32/32 [==============================] - 27s 859ms/step - loss: 0.8994 - acc: 0.7363
Epoch 10/100
32/32 [==============================] - 27s 854ms/step - loss: 0.7825 - acc: 0.7588
Epoch 11/100
32/32 [============

In [9]:
test_generator.reset()
print(model.evaluate_generator(test_generator, steps=5000))

[1.4155961280392264, 0.63971983164771662]


In [10]:
model.save("birds-inceptionv3.model")